# Project Euler

I came to Python programming with some prior experience with Java and C programming, although I haven't written anything in either language in at least two years (C++ is probably my next target language after Python, though). Given the computational nature of my interests (and the fact that I was and remain an entirely self-taught programmer) the [Project Euler problems](https://projecteuler.net/archives) are an ideal way of learning the language.

This notebook contains my solutions to the first **20** Project Euler problems.

In [7]:
import time
import itertools
import functools
import math

## Problem 1

**Statement**

If we list all the natural numbers below 10 that are multiples of 3 or 5, we get 3, 5, 6 and 9. The sum of these multiples is 23. Find the sum of all the multiples of 3 or 5 below 1000.

In [53]:
start_time = time.time()
print(sum(filter(lambda n: (n % 3 == 0 or n % 5 == 0), range(1, 1000))))
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

233168
Executed in <0.001 seconds.


## Problem 2

**Statement**

By considering the terms in the Fibonacci sequence whose values do not exceed four million, find the sum of the even-valued terms.

In [52]:
def fib():
    a = 0
    b = 1
    yield a
    yield b
    while True:
        c = a + b
        a = b
        b = c
        yield c

start_time = time.time()
print(sum([n for n in itertools.takewhile(lambda n: n < 4.0*10**6, fib()) if n % 2 == 0]))
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

4613732
Executed in <0.001 seconds.


## Problem 3

**Statement**

What is the largest prime factor of the number 600851475143?

**Analysis**

The brute force solution is far too slow, instead we use a [Sieve of Eratosthenes](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes). Note the easily verified fact that every potential prime factor $f_a$ of $n$ such that $f \geq \sqrt n$ necessarily has a cofactor $b$ such that $f_b \leq \sqrt n$, so we may stop our sieve there and then check the primality of cofactors to find the largest prime.

List indices begin at 0, not 1. To compensate I elected to add an extra value and then sheer off the first value from the boolean field.

In [455]:
def primes_less_than(mx):
    n = 2
    # Add an extra value to the field to count 0...mx.
    bool_field = [True]*(mx + 1)
    for num in range(2, mx + 1):
        if bool_field[num] == True:
            for mult in range(2 * num, mx, num):
                bool_field[mult] = False
    # Remove 0 and 1 from the results.
    return [n for n in range(len(bool_field)) if bool_field[n] == True][2:]

def is_prime(n):
    cp = primes_less_than(int(math.floor(math.sqrt(n))))
    return True not in map(lambda p: n % p == 0, cp)

def largest_prime_factor(n):
    cp = primes_less_than(int(math.floor(math.sqrt(n))))
    f_a = [p for p in cp if n % p == 0]
    f_b = [int(n / p) for p in f_a if is_prime(int(n / p))]
    if not cp or not f_a:
        return n
    return max(f_a + f_b)

start_time = time.time()
print(largest_prime_factor(600851475143))
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

6857
Executed in <0.580 seconds.


## Problem 4

**Statement**

A palindromic number reads the same both ways. The largest palindrome made from the product of two 2-digit numbers is 9009 = 91 × 99. Find the largest palindrome made from the product of two 3-digit numbers.

In [462]:
def is_palindrome(n):
    return str(n) == str(n)[::-1]

start_time = time.time()
candidates = []
for i in range(600, 999):
    for j in range(600, 999):
        if is_palindrome(i * j):
            candidates.append(i * j)
print(max(candidates))
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

906609
Executed in <0.240 seconds.


## Problem 5

**Statement**

2520 is the smallest number that can be divided by each of the numbers from 1 to 10 without any remainder. What is the smallest positive number that is evenly divisible by all of the numbers from 1 to 20?

**Analysis**

This ought to be rephrased as: "what is the [least common multiple](https://en.wikipedia.org/wiki/Least_common_multiple) of 1, ... , 20?" A basic result from number theory is that such an LCM will be the maximum of the powers of the prime factorizations of the numbers.

Since we are working with small numbers we will use a "dirty" unoptimized prime factorization algorithm.

In [5]:
def factorize(n):
    for i in range(2, math.floor(n / 2)):
        if n % i == 0:
            return [i] + factorize(n / i)
    return [math.floor(n)]

def merge(l1, l2):
    ret = []
    for f in (set(l1) | set(l2)):
        ret += [f for numtimes in range(0, max(l1.count(f), l2.count(f)))]
    return ret

start_time = time.time()
solution_factorized = []
for f in range(20,1,-1):
    solution_factorized = merge(solution_factorized, factorize(f))
print(functools.reduce(lambda x, y: x * y, solution_factorized))
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

232792560
Executed in <0.001 seconds.


## Problem 6

**Statement**

The sum of the squares of the first ten natural numbers is $12 + 22 + ... + 102 = 385$. The square of the sum of the first ten natural numbers is $(1 + 2 + ... + 10)2 = 552 = 3025$. Hence the difference between the sum of the squares of the first ten natural numbers and the square of the sum is $3025 − 385 = 2640$.

Find the difference between the sum of the squares of the first one hundred natural numbers and the square of the sum.

In [519]:
start_time = time.time()
print(abs(sum([n**2 for n in range(0, 100)]) - sum(range(0, 100))**2))
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

24174150
Executed in <0.000 seconds.


## Problem 7

**Statement**

By listing the first six prime numbers: 2, 3, 5, 7, 11, and 13, we can see that the 6th prime is 13. What is the 10,001st prime number?

**Analysis**

The brute force way (first solution) is to count up using the `is_prime()` method we defined earlier, in problem 3.

The smarter way is to use the [prime number theorem](https://en.wikipedia.org/wiki/Prime_number_theorem): this theorem states that the prime numbers are asymptotically distributed, in particular with $p_n \sim n\log{n}$. So we build a sieve that's big enough to capture this number and run that instead!

Math, you crazy.

In [526]:
start_time = time.time()
i = n = 0
while n < 10001:
    if is_prime(i):
        n += 1
        i += 1
    else:
        i += 1
print(i - 1)
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

104723
Executed in <11.529 seconds.


In [533]:
start_time = time.time()
print(primes_less_than(int(15000*math.log(10001)))[9998])
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

104723
Executed in <0.081 seconds.


## Problem 8

**Statement**

Find the 13 adjacent digits in a crazy numeric sequence which have the highest total product.

In [573]:
n = '7316717653133062491922511967442657474235534919493496983520312774506326239578318016984801869478851843858615607891129494954595017379583319528532088055111254069874715852386305071569329096329522744304355766896648950445244523161731856403098711121722383113622298934233803081353362766142828064444866452387493035890729629049156044077239071381051585930796086670172427121883998797908792274921901699720888093776657273330010533678812202354218097512545405947522435258490771167055601360483958644670632441572215539753697817977846174064955149290862569321978468622482839722413756570560574902614079729686524145351004748216637048440319989000889524345065854122758866688116427171479924442928230863465674813919123162824586178664583591245665294765456828489128831426076900422421902267105562632111110937054421750694165896040807198403850962455444362981230987879927244284909188845801561660979191338754992005240636899125607176060588611646710940507754100225698315520005593572972571636269561882670428252483600823257530420752963450'

def prod_digits(num):
    s = 1
    while num:
        s *= (num % 10)
        num /= 10
        num = int(num)
    return s

start_time = time.time()
nums = [int(n[i:i + 13]) for i in range(0, len(n) - 13)]
products = [prod_digits(n) for n in nums]
print(max(products))
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

23514624000
Executed in <0.009 seconds.


## Problem 9

**Statement**

There exists exactly one Pythagorean triplet for which $a + b + c = 1000$. Find the product $abc$.

**Analysis**

Through substitution we know that $a + b + \sqrt{a^2 + b^2} = 1000$. We can find our answer by taking only values of a and b for which the latter value, $\sqrt{a^2 + b^2}$, is a whole number, and then flinging them against 1000.

There is a general property that all $c = 4n + 1$ for some $n$. So we can generate $c$ with which to test $a$ and $b$. It can be shown algebraically that, using the fact that the minimum value that any of the variables can take on is 1, the largest values they can take on are capped:

* $c_{max} = 998$, implying that $n_{max} = 249$.
* b_max given some n_max is demonstrably 998-4n.

We iterate through this constrained range to create our $a$, then test it against the Pythagorean relationship. The first $a$ that proves sufficient is our answer.

In [572]:
start_time = time.time()
for n in range(1,250):
    for b in range(1, 998 - 4*n):
        c = 4*n + 1
        a = 1000 - b - c
        if a**2 + b**2 == c**2:
            print(a*b*c)
            break
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

31875000
Executed in <0.267 seconds.


## Problem 10

**Statement**

Find the sum of all primes below 2 million.

**Analysis**

Solved this back in question number 3!

In [575]:
start_time = time.time()
print(sum(primes_less_than(2*10**6)))
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

142915828922
Executed in <1.356 seconds.


## Problem 11

What is the greatest product of four adjacent numbers in the same direction (up, down, left, right, or diagonally) in the given 20×20 grid?

In [103]:
grid = """08 02 22 97 38 15 00 40 00 75 04 05 07 78 52 12 50 77 91 08
49 49 99 40 17 81 18 57 60 87 17 40 98 43 69 48 04 56 62 00
81 49 31 73 55 79 14 29 93 71 40 67 53 88 30 03 49 13 36 65
52 70 95 23 04 60 11 42 69 24 68 56 01 32 56 71 37 02 36 91
22 31 16 71 51 67 63 89 41 92 36 54 22 40 40 28 66 33 13 80
24 47 32 60 99 03 45 02 44 75 33 53 78 36 84 20 35 17 12 50
32 98 81 28 64 23 67 10 26 38 40 67 59 54 70 66 18 38 64 70
67 26 20 68 02 62 12 20 95 63 94 39 63 08 40 91 66 49 94 21
24 55 58 05 66 73 99 26 97 17 78 78 96 83 14 88 34 89 63 72
21 36 23 09 75 00 76 44 20 45 35 14 00 61 33 97 34 31 33 95
78 17 53 28 22 75 31 67 15 94 03 80 04 62 16 14 09 53 56 92
16 39 05 42 96 35 31 47 55 58 88 24 00 17 54 24 36 29 85 57
86 56 00 48 35 71 89 07 05 44 44 37 44 60 21 58 51 54 17 58
19 80 81 68 05 94 47 69 28 73 92 13 86 52 17 77 04 89 55 40
04 52 08 83 97 35 99 16 07 97 57 32 16 26 26 79 33 27 98 66
88 36 68 87 57 62 20 72 03 46 33 67 46 55 12 32 63 93 53 69
04 42 16 73 38 25 39 11 24 94 72 18 08 46 29 32 40 62 76 36
20 69 36 41 72 30 23 88 34 62 99 69 82 67 59 85 74 04 36 16
20 73 35 29 78 31 90 01 74 31 49 71 48 86 81 16 23 57 05 54
01 70 54 71 83 51 54 69 16 92 33 48 61 43 52 01 89 19 67 48
"""
nums = [chunk.split(maxsplit=20) for chunk in [chunk for chunk in grid.split('\n')]]

def sequences(nums):
    ret = []
    for a, b in itertools.product(range(20), range(20)):
        if a <= 16:
            ret.append([nums[n][b] for n in range(a, a + 4)])
        if b <= 16:
            ret.append([nums[a][n] for n in range(b, b + 4)])
        if a <= 16 and b <= 16:
            ret.append([nums[a][b] for a, b in zip(range(a, a + 4), range(b, b + 4))])
        if a <= 16 and b >= 3:
            ret.append([nums[a][b] for a, b in zip(range(a, a + 4), range(b, b - 4, -1))])
    return ret
            
for a, b in itertools.product(range(20), range(20)):
    nums[a][b] = int(nums[a][b])
start_time = time.time()
print(max([functools.reduce(lambda x, y: x * y, subseq) for subseq in sequences(nums)]))
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

70600674
Executed in <0.010 seconds.


## Problem 12

**Statement**

The sequence of triangle numbers is generated by adding the natural numbers. So the 7th triangle number would be $1 + 2 + 3 + 4 + 5 + 6 + 7 = 28$. The first ten terms would be:

$1, 3, 6, 10, 15, 21, 28, 36, 45, 55, \ldots$

Let us list the factors of the first seven triangle numbers:

     1: 1
     3: 1,3
     6: 1,2,3,6
    10: 1,2,5,10
    15: 1,3,5,15
    21: 1,3,7,21
    28: 1,2,4,7,14,28

We can see that 28 is the first triangle number to have over five divisors.

What is the value of the first triangle number to have over five hundred divisors?

**Analysis**

Per the [divisor function](https://en.wikipedia.org/wiki/Divisor_function), a result from number theory states that the number of divisors of some number $n$ with the prime factorization $\prod_{i=1}^n p_i^{k_i}$ is given by:

$$\tau (n) = (k_1 + 1)(k_2 + 1)(...)(k_n + 1)$$

For example:

$$4200 = 2^33^15^27^1 \rightarrow \tau (4200) = (3+1)(1+1)(2+1)(1+1) = 48$$

This gives us a fast way of counting the number of divisors of a triangle number.

In [7]:
def count_divisors(num):
    f = factorize(num)
    return functools.reduce(lambda x, y: x * y, ([f.count(n) + 1 for n in set(f)]))

def triangle_numbers():
    n = 5
    while True:
        yield sum(range(1,n))
        n += 1

start_time = time.time()
for t_n in triangle_numbers():
   if count_divisors(t_n) > 500:
        print(t_n)
        break
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

76576500
Executed in <7.350 seconds.


## Problem 13

**Statement**

Work out the first ten digits of the sum of the following (obnoxious) one-hundred 50-digit numbers.

In [116]:
grid = """37107287533902102798797998220837590246510135740250
46376937677490009712648124896970078050417018260538
74324986199524741059474233309513058123726617309629
91942213363574161572522430563301811072406154908250
23067588207539346171171980310421047513778063246676
89261670696623633820136378418383684178734361726757
28112879812849979408065481931592621691275889832738
44274228917432520321923589422876796487670272189318
47451445736001306439091167216856844588711603153276
70386486105843025439939619828917593665686757934951
62176457141856560629502157223196586755079324193331
64906352462741904929101432445813822663347944758178
92575867718337217661963751590579239728245598838407
58203565325359399008402633568948830189458628227828
80181199384826282014278194139940567587151170094390
35398664372827112653829987240784473053190104293586
86515506006295864861532075273371959191420517255829
71693888707715466499115593487603532921714970056938
54370070576826684624621495650076471787294438377604
53282654108756828443191190634694037855217779295145
36123272525000296071075082563815656710885258350721
45876576172410976447339110607218265236877223636045
17423706905851860660448207621209813287860733969412
81142660418086830619328460811191061556940512689692
51934325451728388641918047049293215058642563049483
62467221648435076201727918039944693004732956340691
15732444386908125794514089057706229429197107928209
55037687525678773091862540744969844508330393682126
18336384825330154686196124348767681297534375946515
80386287592878490201521685554828717201219257766954
78182833757993103614740356856449095527097864797581
16726320100436897842553539920931837441497806860984
48403098129077791799088218795327364475675590848030
87086987551392711854517078544161852424320693150332
59959406895756536782107074926966537676326235447210
69793950679652694742597709739166693763042633987085
41052684708299085211399427365734116182760315001271
65378607361501080857009149939512557028198746004375
35829035317434717326932123578154982629742552737307
94953759765105305946966067683156574377167401875275
88902802571733229619176668713819931811048770190271
25267680276078003013678680992525463401061632866526
36270218540497705585629946580636237993140746255962
24074486908231174977792365466257246923322810917141
91430288197103288597806669760892938638285025333403
34413065578016127815921815005561868836468420090470
23053081172816430487623791969842487255036638784583
11487696932154902810424020138335124462181441773470
63783299490636259666498587618221225225512486764533
67720186971698544312419572409913959008952310058822
95548255300263520781532296796249481641953868218774
76085327132285723110424803456124867697064507995236
37774242535411291684276865538926205024910326572967
23701913275725675285653248258265463092207058596522
29798860272258331913126375147341994889534765745501
18495701454879288984856827726077713721403798879715
38298203783031473527721580348144513491373226651381
34829543829199918180278916522431027392251122869539
40957953066405232632538044100059654939159879593635
29746152185502371307642255121183693803580388584903
41698116222072977186158236678424689157993532961922
62467957194401269043877107275048102390895523597457
23189706772547915061505504953922979530901129967519
86188088225875314529584099251203829009407770775672
11306739708304724483816533873502340845647058077308
82959174767140363198008187129011875491310547126581
97623331044818386269515456334926366572897563400500
42846280183517070527831839425882145521227251250327
55121603546981200581762165212827652751691296897789
32238195734329339946437501907836945765883352399886
75506164965184775180738168837861091527357929701337
62177842752192623401942399639168044983993173312731
32924185707147349566916674687634660915035914677504
99518671430235219628894890102423325116913619626622
73267460800591547471830798392868535206946944540724
76841822524674417161514036427982273348055556214818
97142617910342598647204516893989422179826088076852
87783646182799346313767754307809363333018982642090
10848802521674670883215120185883543223812876952786
71329612474782464538636993009049310363619763878039
62184073572399794223406235393808339651327408011116
66627891981488087797941876876144230030984490851411
60661826293682836764744779239180335110989069790714
85786944089552990653640447425576083659976645795096
66024396409905389607120198219976047599490197230297
64913982680032973156037120041377903785566085089252
16730939319872750275468906903707539413042652315011
94809377245048795150954100921645863754710598436791
78639167021187492431995700641917969777599028300699
15368713711936614952811305876380278410754449733078
40789923115535562561142322423255033685442488917353
44889911501440648020369068063960672322193204149535
41503128880339536053299340368006977710650566631954
81234880673210146739058568557934581403627822703280
82616570773948327592232845941706525094512325230608
22918802058777319719839450180888072429661980811197
77158542502016545090413245809786882778948721859617
72107838435069186155435662884062257473692284509516
20849603980134001723930671666823555245252804609722
53503534226472524250874054075591789781264330331690"""
nums = list(map(lambda x: int(x), grid.split('\n')))
start_time = time.time()
print(str(sum(nums))[:10])
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

5537376230
Executed in <0.000 seconds.


## Problem 14

**Statement**

Which starting number, under one million, produces the longest [Collatz sequence](https://en.wikipedia.org/wiki/Collatz_conjecture)?

**Analysis**

The obvious brute force solution is to create a collatz algorithm and run that on every number from 1 to 1,000,000, but this takes minutes to process. We achieve a significant speedup by caching past results along the way.

In [18]:
def collatz_stepper(n):
    while True:
        if n == 1:
            yield StopIteration
        elif n % 2 == 0:
            n = n / 2
            yield int(n)
        else:
            n = int(3*n + 1)
            yield n
    
def collatz_number_cached(n, cache):
    collatz_number = 0
    stp = collatz_stepper(n)
    for cn in stp:
        collatz_number += 1
        if cn == 1:
            return collatz_number
        if cn in cache.keys():
            collatz_number += cache[cn]
            return collatz_number
        else:
            continue        

cache = dict()
max_collatz_terminand = 0
max_collatz_number = 0
start_time = time.time()
for n in range(2, 10**6):
    collatz_number = collatz_number_cached(n, cache)
    cache[n] = collatz_number
    if collatz_number > max_collatz_number:
        max_collatz_number = collatz_number
        max_collatz_terminand = n
print(max_collatz_terminand)
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

837799
Executed in <6.370 seconds.


## Problem 15

**Statement**

Starting in the top left corner of a 2×2 grid, and only being able to move to the right and down, there are exactly 6 routes to the bottom right corner. How many such routes are there through a 20×20 grid?

**Analysis**

This is a simple combinatorical problem. The solution is $40 \choose{20}$.

In [19]:
def nCr(n,r):
    f = math.factorial
    return f(n) / f(r) / f(n-r)

start_time = time.time()
print(nCr(40, 20))
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

137846528820.0
Executed in <0.000 seconds.


## Problem 16

**Statement**

$2^{15} = 32768$ and the sum of its digits is $3 + 2 + 7 + 6 + 8 = 26$. What is the sum of the digits of the number $2^{1000}$?

**Analysis**

There's no way to know this answer for sure without knowing all of the digits of the summation, but getting $2^{1000}$ isn't a problem: [squared exponentiation](https://en.wikipedia.org/wiki/Exponentiation_by_squaring) is supremely efficient.

In [47]:
start_time = time.time()
print(sum(map(lambda n: int(n), list(str(2**1000)))))
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

1366
Executed in <0.000 seconds.


## Problem 17

**Statement**

If the numbers 1 to 5 are written out in words: one, two, three, four, five, then there are 3 + 3 + 5 + 4 + 4 = 19 letters used in total. If all the numbers from 1 to 1000 (one thousand) inclusive were written out in words, how many letters would be used? Do not count spaces or hyphens. For example, 342 (three hundred and forty-two) contains 23 letters and 115 (one hundred and fifteen) contains 20 letters. The use of "and" when writing out numbers is in compliance with British usage.

**Analysis**

Another string interpretation problem...

In [84]:
units = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']
unitary_tens = ['ten', 'eleven', 'twelve', 'thirteen', 'fourteen', 'fifteen', 'sixteen', 'seventeen', 'eighteen', 'nineteen']
tens = ['ten', 'twenty', 'thirty', 'forty', 'fifty', 'sixty', 'seventy', 'eighty', 'ninety']
places = ['hundred']

def string_repr(n):
    if n == 0:
        return ''
    elif n < 10:
        return units[n - 1]
    elif n < 20:
        return unitary_tens[n - 10]
    elif n < 100:
        return tens[math.floor(n / 10) - 1] + ' ' + string_repr(n % 10)
    elif n < 1000:
        return units[math.floor(n / 100) - 1] + ' hundred and ' + string_repr(n % 100)

start_time = time.time()
s = [string_repr(n) for n in range(1, 999)] + ['thousand']
tot = 0
for n in s:
    n = n.replace(' ', '')
    # print(n)
    tot += len(n)
print(tot)
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

21124
Executed in <0.004 seconds.


## Problem 18

**Statement**

Find the heighest-weighted total from amongst those present in the graph.

**Note**

I'll come back to this one later. Would be fun to try a hill-climber algorithm solution.

## Problem 19

**Statement**

    1 Jan 1900 was a Monday.
    Thirty days has September,
    April, June and November.
    All the rest have thirty-one,
    Saving February alone,
    Which has twenty-eight, rain or shine.
    And on leap years, twenty-nine.
    A leap year occurs on any year evenly divisible by 4, but not on a century unless it is divisible by 400.

How many Sundays fell on the first of the month during the twentieth century (1 Jan 1901 to 31 Dec 2000)?

**Analysis**

Wacky. This is a hand implementation, it would be easier to use the `arrow` package or the [Gaussian formula](https://en.wikipedia.org/wiki/Determination_of_the_day_of_the_week) but I wanted to do it by hand.

In [133]:
class Date():
    day = 6
    month = 1
    year = 1900
    
    def __init__(self):
        pass
    
    def __repr__(self):
        return str((self.year, self.month, self.day))
    
    def jump(self):
        if self.month not in (2, 4, 6, 9, 11):
            if self.day <= (31 - 7):
                self.day += 7
            else:
                self.day = self.day + 7 - 31
                if self.month <= 11:
                    self.month += 1
                else:
                    self.month = 1
                    self.year += 1
        elif self.month in (4, 6, 9, 11):
            if self.day <= (30 - 7):
                self.day += 7
            else:
                self.day = self.day + 7 - 30
                self.month += 1
        elif self.month == 2:
            if self.day <= (28 - 7):
                self.day += 7
            elif (self.year % 4 == 0 and self.year % 100 != 0) or (self.year % 4 == 0 and self.year % 400 == 0):
                if self.day == 22:
                    self.day += 7
                else:
                    self.day = self.day + 7 - 29
                    self.month += 1
            else:
                self.day = self.day + 7 - 28
                self.month += 1

start_time = time.time()
d = Date()
first_day_saturdays = []
while d.year < 2001:
    d.jump()
    if d.day == 1:
        first_day_saturdays += [str(d)]
print(len(first_day_saturdays))
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

173
Executed in <0.007 seconds.


## Problem 20

**Statement**

$n!$ means $n × (n − 1) × ... × 3 × 2 × 1$ For example, $10! = 10 × 9 × ... × 3 × 2 × 1 = 3628800$, and the sum of the digits in the number $10!$ is $3 + 6 + 2 + 8 + 8 + 0 + 0 = 27$. Find the sum of the digits in the number $100!$

In [88]:
start_time = time.time()
print(sum([int(d) for d in list(str(math.factorial(100)))]))
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

648
Executed in <0.000 seconds.
